# SQLite with Python
This notebook provides a minimalistic tutorial for performing basic SQLite operations with Python. Typically one would install and use MySQL for a proper production system, however SQLite is close enough for learning how relational databases work.

## Basic Example

In [1]:
# pip install openpyxl
import sqlite3
import pandas as pd
import traceback
import openpyxl
import uuid
import json

### Connect to the database

To reset the database, simply restart the python kernel and delete the [sqlite_database.db](sqlite_database.db) file.

In [2]:
database_file_path = 'sqlite_database.db'
connection = sqlite3.connect(database_file_path)

Remove old data if it exists (in case you ran the tutorial before).

In [3]:
connection.execute("DROP TABLE IF EXISTS users;")
connection.execute("DROP TABLE IF EXISTS users_backup;")

### Create a table (Like in MS Excel)

SQLite and MySQL are built on the SQL language. SQL stands for Structured Query Language. A query is an instruction that is sent to the database. The one below creates a new table to contain information about the users of the new system. You can read more about SQL queries and the syntax [here](https://www.w3schools.com/sql/sql_syntax.asp), or simply copy and modify the code below to create additional tables.

In the query below, "users" specifies the name of the tabe, the first value on each row is the column name, the second is the data type, "NOT NULL" means the database will not allow a user row to be added to the table without that value being specified.

A "PRIMARY KEY" is the string or number that uniquely identifies each row in the table. Think of it as your social security number. There can never be 2 users with the same id, meaning it is unique.

"AUTOINCREMENT" means that the value of the id will be automatically incremented by 1 each time a new row is added to the table. This is useful because you don't have to worry about specifying the id when adding a new user.

The "UNIQUE" constraint means that the database will refuse to accept new entries into this table if there already exists a row with the same value for the specified column. The UNIQUE constraint may be added to the list of constraints following the name of a single column, or can be used independently. Below it is used to prevent someone from registering two users with the same combination of name and email address. In other words, different people can have the same name, and the same person (identified by his or her email) can have multiple users with different usernames.

A list of SQL constraints can be found [here](https://www.w3schools.com/sql/sql_constraints.asp).

In [4]:
# Use the cursor to create a new table in the database
connection.execute("""
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        email TEXT NOT NULL,
        password TEXT NOT NULL,
        UNIQUE (name, email)
    );
""")

### Insert Rows

Insert 2 new user rows into the users table

In [5]:
connection.execute("INSERT INTO users (name, email, password) VALUES ('Alice', 'alice@example.com', '1234');")
connection.execute("INSERT INTO users (name, email, password) VALUES ('John', 'john@example.com', '5678');")

### List Rows

List all columns for all the users currently in the users table. Notice how the id column is automatically incremented by 1 each time a new row is added.

In [6]:
users = connection.execute("SELECT * FROM users;").fetchall()
users

[(1, 'Alice', 'alice@example.com', '1234'),
 (2, 'John', 'john@example.com', '5678')]

### List Specific Columns

List only the name and email columns for all the users currently in the users table

In [7]:
users = connection.execute("SELECT name, email FROM users;").fetchall()
users

[('Alice', 'alice@example.com'), ('John', 'john@example.com')]

### Filter Rows

Find alice's email

In [8]:
emails = connection.execute("SELECT email FROM users WHERE name = 'Alice';").fetchall()

print("Result:", emails)
print("Alice's Email:", emails[0][0])

Result: [('alice@example.com',)]
Alice's Email: alice@example.com


### Update Rows

Update Alice's email by setting the value of the "email" column in every row of the "users" table where the value of the "name" column is equal to the string "Alice".

In [9]:
connection.execute("UPDATE users SET email = 'alice.new@example.com' WHERE name = 'Alice';")

# Print her email again
connection.execute("SELECT email FROM users WHERE name = 'Alice';").fetchall()

[('alice.new@example.com',)]

### Parameterized Queries

Usually when querying a database, one uses information received by the user to do so. For example, when logging into a service, the user provides an email that is used to find the password in the database. In order to use a variable as part of the query, one can use a parameterized query. This is done by using a question mark (?) as a placeholder for the variable. The variable is then passed as a tuple to the execute function.

In [10]:
# Variable from user input
username = 'Alice'

parameters = (username,) # The last comma is important!

password = connection.execute("""
    SELECT password FROM users WHERE name = ?;
""", parameters).fetchall()

password

[('1234',)]

### SQL Injection Attacks

Why not simply use f-strings to insert variables into the query? This is because of SQL injection attacks. If a user provides a string that contains a semicolon, followed by a malicious query, the database will execute the malicious query. Watch below, as a hacker manages to "inject" a query into the username.

In [11]:
# Variable from user input
username = "'Alice'; UPDATE users SET password = 'hacked' WHERE name = 'Alice';"

password = connection.executescript(f"""
    SELECT password FROM users WHERE name = {username};
""").fetchall()

password

[]

Now lets see what her new password is.

In [12]:
connection.execute("SELECT password FROM users WHERE name = 'Alice';").fetchall()

[('hacked',)]

Looks like the injection attack worked, and Alice's account is now stolen. In theory the attacker could delete the entire database this way, which is the origin of [these](https://www.google.com/search?tbm=isch&q=license+plate+sql+injection) memes.

You may have noticed that ```connection.executescript()``` was used instead of ```connection.execute()```. Run the code below to see why. 

In [13]:
# Variable from user input
username = "'Alice'; UPDATE users SET password = 'hacked' WHERE name = 'Alice';"

try:
    password = connection.execute(f"SELECT password FROM users WHERE name = {username};").fetchall()
except sqlite3.ProgrammingError as e:
    traceback.print_exc()

Traceback (most recent call last):
  File "C:\Users\noskn\AppData\Local\Temp\ipykernel_36880\3039779683.py", line 5, in <module>
    password = connection.execute(f"SELECT password FROM users WHERE name = {username};").fetchall()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
sqlite3.ProgrammingError: You can only execute one statement at a time.


As you can see, ```connection.execute()``` will not allow queries that contain multiple statements. This is a security feature that prevents SQL injection attacks. Does this mean using the correct function keeps you safe?

In [14]:
# Variable from user input
username = "Alice"
password = "incorrect password' OR '1' = '1"

def check_login(username, password):
    user = connection.execute(f"""
        SELECT * FROM users WHERE name = '{username}' AND password = '{password}';
    """).fetchall()
    
    if len(user) == 0:
        print("Login failed")
        return None
    else:
        print("Login successful")
        return user

user = check_login(username, password)
user

Login successful


[(1, 'Alice', 'alice.new@example.com', 'hacked'),
 (2, 'John', 'john@example.com', '5678')]

In this case, the attacker injected a filter into the password, allowing him to bypass the login. Lets try to fix this by using a parameterized query as well as the more secure ```connection.execute()``` function.

In [15]:
# Variable from user input
username = "Alice"
password = "incorrect password' OR '1' = '1"

def check_login(username, password):
    parameters = (username, password)
    
    user = connection.execute(f"""
        SELECT * FROM users WHERE name = ? AND password = ?;
    """, parameters).fetchall()
    
    print(user)
    
    if len(user) == 0:
        print("Login failed")
        return None
    else:
        print("Login successful")
        return user

user = check_login(username, password)
user

[]
Login failed


The library ensures that the parameters are interpreted as strings and strings only.

## Non-Sequential IDs.

Previously, AUTOINCREMENT was used to automatically increment the id by 1 each time a new row was added. This is simple, however it also creates a vulnerability. What if there is a piece of code that allows anyone to fetch information as long as they know the id? An example of this could be unlisted youtube videos and google documents, where anyone with the link can access the content. If i have a video with the id=10, i know i can probably just fetch every video with an id less than 10 to find other unlisted videos. One of the best ways to avoid this issue is to use UUIDs. UUIDs are standardized long strings of alphanumeric characters, and they are used because there are a lot of possible UUID's. 2^128 to be exact. This means that the chances of guessing a valid UUID are extremely low.

Lets modify the existing table to use UUIDs instead of AUTOINCREMENT.

In [16]:
# Create a new table to contain the old users table
connection.execute("""
    CREATE TABLE users_backup AS SELECT * FROM users;
""")

# Delete the old users table
connection.execute("""
    DROP TABLE users;
""")

# Create a new users table where the id is a string and no AUTOINCREMENT
connection.execute("""
    CREATE TABLE users (
        id TEXT PRIMARY KEY,
        name TEXT NOT NULL,
        email TEXT NOT NULL,
        password TEXT NOT NULL,
        UNIQUE (name, email)
    );
""")

# Copy the data from the backup table to the new table
connection.execute("""
    INSERT INTO users (id, name, email, password) SELECT id, name, email, password FROM users_backup;
""")

# Delete the backup table
connection.execute("""
    DROP TABLE users_backup;
""")

# List the contents of the new users table
connection.execute("""
    SELECT * FROM users;
""").fetchall()

[('1', 'Alice', 'alice.new@example.com', 'hacked'),
 ('2', 'John', 'john@example.com', '5678')]

The ID column is now TEXT instead of INTEGER, as can be seen by the 'quotes' around each value. Let's create 3 new users using UUIDv4.

In [17]:
def create_user(name, email, password):
    user_id = str(uuid.uuid4())
    
    connection.execute(f"""
        INSERT INTO users (id, name, email, password) VALUES (?, ?, ?, ?);
    """, (user_id, name, email, password))
    
create_user(name='Bob', email='bob@example.com', password='1234')
create_user(name='James', email='james@example.com', password='7654')
create_user(name='Marie', email='marie@example.com', password='0000')

# List the users
connection.execute("SELECT * FROM users;").fetchall()

[('1', 'Alice', 'alice.new@example.com', 'hacked'),
 ('2', 'John', 'john@example.com', '5678'),
 ('dc61e78b-673d-4a51-ad37-ab6ff4ccb858', 'Bob', 'bob@example.com', '1234'),
 ('b8f2b608-5a6a-44e1-80e4-ba1d718d7490',
  'James',
  'james@example.com',
  '7654'),
 ('00dd4d71-37bc-4f05-8631-c2f5029615db',
  'Marie',
  'marie@example.com',
  '0000')]

Good luck guessing the user id now!

## Relationships

SQL databases are relational, meaning that rows from different tables can be related to each other. There are mechanisms to enforce these relationships, such as the "FOREIGN KEY" constraint, however the simpler way to do this is to make sure every row has an id, and then refer to the id through a dedicated column in other tables.

Lets create a table of devices, and then a second relational table to connect users to those devices. For the device table, lets also add a column for a "created_at" timestamp to keep track of when devices are added to the system. The "DEFAULT" value of this column (if not specified when inserting a new row) will be now. Also known in SQL as "CURRENT_TIMESTAMP".

In [18]:
connection.execute("""
    CREATE TABLE IF NOT EXISTS devices (
        id TEXT PRIMARY KEY,
        name TEXT NOT NULL,
        created_at TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP
    );
""")

connection.execute("""
    CREATE TABLE IF NOT EXISTS user_devices (
        user_id TEXT NOT NULL,
        device_id TEXT NOT NULL,
        PRIMARY KEY (user_id, device_id)
    );
""")

Now lets add some devices to the table.

In [19]:
james_laptop_id = str(uuid.uuid4())
james_phone_id = str(uuid.uuid4())
maries_laptop_id = str(uuid.uuid4())
maries_phone_id = str(uuid.uuid4())

connection.execute(f"INSERT INTO devices (id, name) VALUES ('{james_laptop_id}', 'James Laptop');")
connection.execute(f"INSERT INTO devices (id, name) VALUES ('{james_phone_id}', 'James Phone');")
connection.execute(f"INSERT INTO devices (id, name) VALUES ('{maries_laptop_id}', 'Maries Laptop');")
connection.execute(f"INSERT INTO devices (id, name) VALUES ('{maries_phone_id}', 'Maries Phone');")
connection.execute(f"SELECT * FROM devices;").fetchall()

[('1bd4e3b3-deb0-47e4-a432-f7ec09c1861b',
  'James Laptop',
  '2024-08-30 20:44:53'),
 ('9ce9625e-f46f-458c-9147-a73c3ec80f11',
  'James Phone',
  '2024-08-30 20:44:53'),
 ('2c3d6fe0-e375-4b6f-a528-de76b09bbca9',
  'Maries Laptop',
  '2024-08-30 20:44:53'),
 ('6ed9c06a-a4bf-46e5-bf2a-4aa4b4704035',
  'Maries Phone',
  '2024-08-30 20:44:53')]

And relate those devices to their respective users. Lets use a SELECT statement to get james' id. Yes, you can use query results as values for INSERT statements. They are called subqueries and must be wrapped in (parentheses).

In [20]:
james_user_id_query = "SELECT id FROM users WHERE name = 'James'"
connection.execute(f"INSERT INTO user_devices (user_id, device_id) VALUES (({james_user_id_query}), '{james_laptop_id}');")
connection.execute(f"INSERT INTO user_devices (user_id, device_id) VALUES (({james_user_id_query}), '{james_phone_id}');")

maries_user_id_query = "SELECT id FROM users WHERE name = 'Marie'"
connection.execute(f"INSERT INTO user_devices (user_id, device_id) VALUES (({maries_user_id_query}), '{maries_laptop_id}');")
connection.execute(f"INSERT INTO user_devices (user_id, device_id) VALUES (({maries_user_id_query}), '{maries_phone_id}');")

connection.execute(f"SELECT * FROM user_devices;").fetchall()

[('b8f2b608-5a6a-44e1-80e4-ba1d718d7490',
  '1bd4e3b3-deb0-47e4-a432-f7ec09c1861b'),
 ('b8f2b608-5a6a-44e1-80e4-ba1d718d7490',
  '9ce9625e-f46f-458c-9147-a73c3ec80f11'),
 ('00dd4d71-37bc-4f05-8631-c2f5029615db',
  '2c3d6fe0-e375-4b6f-a528-de76b09bbca9'),
 ('00dd4d71-37bc-4f05-8631-c2f5029615db',
  '6ed9c06a-a4bf-46e5-bf2a-4aa4b4704035')]

This is hardly human-readable, however to a computer this is just fine. Now lets list all the devices that belong to James by using a JOIN statement. The code below does the following:
1. Selects the users "name" and "email", devices "name" and "created_at", and users "id" and devices "id" columns, and creates the variables "user_name", "user_email", "device_name", "device_created_at", "user_id", and "device_id" respectively.
2. Reads each row in the "devices" table.
3. Finds the entries in the "user_devices" table that have a "device_id" matching the "id" column.
4. Excludes "user_devices" rows that had no match in "devices".
5. Finds the entries in the "users" table that have an "id" matching the "user_id" column.
6. Excludes "users" rows that had no match in the overlapping "users" and "user_devices".
7. Filters the results to only include rows where the "name" column in the "users" table is equal to "James"



In [21]:
def list_devices(username):
    return connection.execute("""
        SELECT
            users.name AS user_name,
            users.email AS email,
            devices.name AS device_name,
            devices.created_at AS device_created,
            users.id AS user_id,
            devices.id AS device_id
        FROM devices
        JOIN user_devices ON devices.id = user_devices.device_id
        JOIN users ON users.id = user_devices.user_id
        WHERE users.name = ?;
    """, (username,))

list_devices(username='James').fetchall()

[('James',
  'james@example.com',
  'James Laptop',
  '2024-08-30 20:44:53',
  'b8f2b608-5a6a-44e1-80e4-ba1d718d7490',
  '1bd4e3b3-deb0-47e4-a432-f7ec09c1861b'),
 ('James',
  'james@example.com',
  'James Phone',
  '2024-08-30 20:44:53',
  'b8f2b608-5a6a-44e1-80e4-ba1d718d7490',
  '9ce9625e-f46f-458c-9147-a73c3ec80f11')]

It works! But how can we extract the variables that were created in the SELECT statement? Read the next section to find out.

## Extracting Column names from query results and Pandas Dataframes

The return value of ```connection.execute()``` is called a cursor, and it holds information about the query.
So far, only the ```cursor.fetchall()``` method has been used to extract the results of the query. There is however also a ```cursor.description``` variable that contains the names of the returned values. The first value of each tuple is the name of the column.

In [22]:
cursor = list_devices(username='James')
cursor.description

(('user_name', None, None, None, None, None, None),
 ('email', None, None, None, None, None, None),
 ('device_name', None, None, None, None, None, None),
 ('device_created', None, None, None, None, None, None),
 ('user_id', None, None, None, None, None, None),
 ('device_id', None, None, None, None, None, None))

Extract the column names.

In [23]:
columns = [desc[0] for desc in cursor.description]
columns

['user_name', 'email', 'device_name', 'device_created', 'user_id', 'device_id']

And the values.

In [24]:
values = cursor.fetchall()
values

[('James',
  'james@example.com',
  'James Laptop',
  '2024-08-30 20:44:53',
  'b8f2b608-5a6a-44e1-80e4-ba1d718d7490',
  '1bd4e3b3-deb0-47e4-a432-f7ec09c1861b'),
 ('James',
  'james@example.com',
  'James Phone',
  '2024-08-30 20:44:53',
  'b8f2b608-5a6a-44e1-80e4-ba1d718d7490',
  '9ce9625e-f46f-458c-9147-a73c3ec80f11')]

Lets make this information easier to manage using a pandas dataframe.

In [25]:
james_devices_df = pd.DataFrame(values, columns=columns)
james_devices_df

,user_name,email,device_name,device_created,user_id,device_id
0,James,james@example.com,James Laptop,2024-08-30 20:44:53,b8f2b608-5a6a-44e1-80e4-ba1d718d7490,1bd4e3b3-deb0-47e4-a432-f7ec09c1861b
1,James,james@example.com,James Phone,2024-08-30 20:44:53,b8f2b608-5a6a-44e1-80e4-ba1d718d7490,9ce9625e-f46f-458c-9147-a73c3ec80f11


Lets make a function to handle this automatically.

In [26]:
def frame_cursor(cursor):
    columns = [desc[0] for desc in cursor.description]
    values = cursor.fetchall()
    return pd.DataFrame(values, columns=columns)

In [27]:
maries_devices = frame_cursor(list_devices(username='Marie'))
maries_devices

,user_name,email,device_name,device_created,user_id,device_id
0,Marie,marie@example.com,Maries Laptop,2024-08-30 20:44:53,00dd4d71-37bc-4f05-8631-c2f5029615db,2c3d6fe0-e375-4b6f-a528-de76b09bbca9
1,Marie,marie@example.com,Maries Phone,2024-08-30 20:44:53,00dd4d71-37bc-4f05-8631-c2f5029615db,6ed9c06a-a4bf-46e5-bf2a-4aa4b4704035


Now the data is easy to manipulate using regular pandas functions.

In [28]:
maries_devices['device_name']

0    Maries Laptop
1     Maries Phone
Name: device_name, dtype: object

## Transactions and Summary

Some times, it is necessary to make changes to multiple tables to have a valid database entry. For example, one must add both a "user_devices" entry as well as a "devices" entry for the last one to be valid. What happens if the "devices" entry is added, but something goes wrong when adding the "user_devices" entry? To ensure information is not partially added, one can use transactions.

In [29]:
# Create a transaction
cursor = connection.cursor()

elizabeth_id = str(uuid.uuid4())
elizabeth_phone_id = str(uuid.uuid4())

# Insert a new user
cursor.execute("INSERT INTO users (id, name, email, password) VALUES (?, ?, ?, ?);",
    (elizabeth_id, 'Elizabeth', 'elizabeth@example.com', '1234'))

# Insert a new device
cursor.execute("INSERT INTO devices (id, name) VALUES (?, ?);",
    (elizabeth_phone_id, 'Elizabeth Phone'))

# Insert the user device relationship
cursor.execute("INSERT INTO user_devices (user_id, device_id) VALUES (?, ?);",
    (elizabeth_id, elizabeth_phone_id))

# List Elizabeth's devices
frame_cursor(list_devices(username='Elizabeth'))

,user_name,email,device_name,device_created,user_id,device_id
0,Elizabeth,elizabeth@example.com,Elizabeth Phone,2024-08-30 20:44:53,0ca93008-7b13-4023-ae34-950de4b3885b,3e6245e3-c567-4bcb-ab5f-0dc70a7ae7e1


Now lets pretend like something went wrong (perhaps the phone_id was alredy taken and the database threw an error). By rolling back the changes using the transaction, the database is left in the same state as before the transaction was started.

In [30]:
# Roll back the change
connection.rollback()

In [31]:
try:
    
    frame_cursor(list_devices(username='Elizabeth'))
    
except sqlite3.OperationalError as e:
    traceback.print_exc()

Traceback (most recent call last):
  File "C:\Users\noskn\AppData\Local\Temp\ipykernel_36880\1327403853.py", line 3, in <module>
    frame_cursor(list_devices(username='Elizabeth'))
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\noskn\AppData\Local\Temp\ipykernel_36880\4076249949.py", line 2, in list_devices
    return connection.execute("""
           ^^^^^^^^^^^^^^^^^^^^^^
sqlite3.OperationalError: no such table: devices


WHAT! No such table???

In [32]:
# List Tables
frame_cursor(connection.execute("SELECT name FROM sqlite_master WHERE type='table';"))

,name
0,sqlite_sequence
1,users_backup
2,users


In [33]:
# List Users
frame_cursor(connection.execute("SELECT * FROM users;"))

,id,name,email,password


In [34]:
frame_cursor(connection.execute("SELECT * FROM users_backup;"))

,id,name,email,password
0,1,Alice,alice.new@example.com,hacked
1,2,John,john@example.com,5678


NOOOOOOOO!!!! The whole time we were in a single transaction and nothing since then has been saved????
Lets remember this for the next time. Always call ```connection.commit()``` after finishing a change to the database.

Lets learn from this misstake and start over.

Clear all tables.

In [35]:
connection.execute("DROP TABLE IF EXISTS devices;")
connection.execute("DROP TABLE IF EXISTS users;")
connection.execute("DROP TABLE IF EXISTS users_backup;")

Create new tables.

In [36]:
# Create users table
connection.execute("""
    CREATE TABLE IF NOT EXISTS users (
        id TEXT PRIMARY KEY,
        name TEXT NOT NULL,
        email TEXT NOT NULL,
        password TEXT NOT NULL,
        UNIQUE (name, email)
    );
""")

# Create devices table
connection.execute("""
    CREATE TABLE IF NOT EXISTS devices (
        id TEXT PRIMARY KEY,
        name TEXT NOT NULL,
        created_at TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP
    );
""")

# Create user_devices table
connection.execute("""
    CREATE TABLE IF NOT EXISTS user_devices (
        user_id TEXT NOT NULL,
        device_id TEXT NOT NULL,
        PRIMARY KEY (user_id, device_id)
    );
""")

Create some helper functions.

In [37]:
def create_user(name, email, password):
    user_id = str(uuid.uuid4())
    
    # No rollbacks needed if this feils because no changes would be made
    connection.execute(f"""
        INSERT INTO users (id, name, email, password) VALUES (?, ?, ?, ?);
    """, (user_id, name, email, password))
    
    # Commit the transaction
    # This saves the changes that were just made to the database
    connection.commit()
    
    return user_id

In [38]:
def create_device(name, user_id):
    device_id = str(uuid.uuid4())
    
    # Create a transaction
    cursor = connection.cursor()
    
    try:
        # Add the device
        cursor.execute(f"""
            INSERT INTO devices (id, name) VALUES (?, ?);
        """, (device_id, name))
    
        # Add the relation
        cursor.execute(f"""
            INSERT INTO user_devices (user_id, device_id) VALUES (?, ?);
        """, (user_id, device_id))
        
        # Commit the change if everything works as expected
        connection.commit()
    except sqlite3.Error as e:
        # Revert the changes if there was an error
        # to avoid a device being created without a relation
        connection.rollback()
        # Rethrow the error after reverting the changes
        raise e
    
    return device_id

In [39]:
def list_users():
    return frame_cursor(connection.execute("SELECT * FROM users;"))

In [40]:
def list_devices():
    return frame_cursor(connection.execute("SELECT * FROM devices;"))

In [41]:
def list_user_devices_table():
    return frame_cursor(connection.execute("SELECT * FROM user_devices;"))

In [42]:
def list_devices_for_user(user_id):
    return frame_cursor(connection.execute("""
        SELECT
            users.name AS user_name,
            users.email AS email,
            devices.name AS device_name,
            devices.created_at AS device_created,
            users.id AS user_id,
            devices.id AS device_id
        FROM devices
        JOIN user_devices ON devices.id = user_devices.device_id
        JOIN users ON users.id = user_devices.user_id
        WHERE users.id = ?;
    """, (user_id,)))

In [43]:
def list_everything_joined():
    return frame_cursor(connection.execute("""
        SELECT *
        FROM users
        JOIN user_devices ON users.id = user_devices.user_id
        JOIN devices ON devices.id = user_devices.device_id;
    """))

In [44]:
bob_user_id = create_user(name='Bob', email='bob@example.com', password='1234')
create_device('Bob Laptop', bob_user_id)
create_device('Bob Phone', bob_user_id)

james_user_id = create_user(name='James', email='james@example.com', password='7654')
create_device('James Laptop', james_user_id)
create_device('James Phone', james_user_id)

marie_user_id = create_user(name='Marie', email='marie@example.com', password='0000')
create_device('Marie Laptop', marie_user_id)
create_device('Marie Phone', marie_user_id)

print("Done!")

Done!


In [45]:
# List the users
list_users()

,id,name,email,password
0,f84c183c-a3f0-4f02-bc06-acd91ca1d048,Bob,bob@example.com,1234
1,c758ce01-fa8b-4f4f-8856-a2c79d1a636c,James,james@example.com,7654
2,4b7706f1-d702-4e63-ba5f-64909819b67d,Marie,marie@example.com,0000


In [46]:
# List the devices
list_devices()

,id,name,created_at
0,ec1846b5-6b82-4e87-b553-cf0c8e36fe61,Bob Laptop,2024-08-30 20:44:54
1,b373cf29-54d1-497a-9843-e81fce68a2b0,Bob Phone,2024-08-30 20:44:54
2,7ff7cb19-5356-4d73-8155-bea6484e7562,James Laptop,2024-08-30 20:44:54
3,3e139f88-e54a-419d-ba9c-84071115da19,James Phone,2024-08-30 20:44:54
4,f53e7bc8-4225-4a86-a937-80e66779a1aa,Marie Laptop,2024-08-30 20:44:54
5,ca614e6e-bd7c-4f81-b95b-9527a84b9aeb,Marie Phone,2024-08-30 20:44:54


In [47]:
# List Relationships
list_user_devices_table()

,user_id,device_id
0,f84c183c-a3f0-4f02-bc06-acd91ca1d048,ec1846b5-6b82-4e87-b553-cf0c8e36fe61
1,f84c183c-a3f0-4f02-bc06-acd91ca1d048,b373cf29-54d1-497a-9843-e81fce68a2b0
2,c758ce01-fa8b-4f4f-8856-a2c79d1a636c,7ff7cb19-5356-4d73-8155-bea6484e7562
3,c758ce01-fa8b-4f4f-8856-a2c79d1a636c,3e139f88-e54a-419d-ba9c-84071115da19
4,4b7706f1-d702-4e63-ba5f-64909819b67d,f53e7bc8-4225-4a86-a937-80e66779a1aa
5,4b7706f1-d702-4e63-ba5f-64909819b67d,ca614e6e-bd7c-4f81-b95b-9527a84b9aeb


In [48]:
# List Bob's devices
list_devices_for_user(bob_user_id)

,user_name,email,device_name,device_created,user_id,device_id
0,Bob,bob@example.com,Bob Phone,2024-08-30 20:44:54,f84c183c-a3f0-4f02-bc06-acd91ca1d048,b373cf29-54d1-497a-9843-e81fce68a2b0
1,Bob,bob@example.com,Bob Laptop,2024-08-30 20:44:54,f84c183c-a3f0-4f02-bc06-acd91ca1d048,ec1846b5-6b82-4e87-b553-cf0c8e36fe61


In [49]:
# List James's devices
list_devices_for_user(james_user_id)

,user_name,email,device_name,device_created,user_id,device_id
0,James,james@example.com,James Phone,2024-08-30 20:44:54,c758ce01-fa8b-4f4f-8856-a2c79d1a636c,3e139f88-e54a-419d-ba9c-84071115da19
1,James,james@example.com,James Laptop,2024-08-30 20:44:54,c758ce01-fa8b-4f4f-8856-a2c79d1a636c,7ff7cb19-5356-4d73-8155-bea6484e7562


In [50]:
# List Marie's devices
list_devices_for_user(marie_user_id)

,user_name,email,device_name,device_created,user_id,device_id
0,Marie,marie@example.com,Marie Phone,2024-08-30 20:44:54,4b7706f1-d702-4e63-ba5f-64909819b67d,ca614e6e-bd7c-4f81-b95b-9527a84b9aeb
1,Marie,marie@example.com,Marie Laptop,2024-08-30 20:44:54,4b7706f1-d702-4e63-ba5f-64909819b67d,f53e7bc8-4225-4a86-a937-80e66779a1aa


In [51]:
# Export Marie's devices to dict
maries_devices_dict = list_devices_for_user(marie_user_id).to_dict(orient='records')

# Pretty print it as JSON
print(json.dumps(maries_devices_dict, indent=4))

[
    {
        "user_name": "Marie",
        "email": "marie@example.com",
        "device_name": "Marie Phone",
        "device_created": "2024-08-30 20:44:54",
        "user_id": "4b7706f1-d702-4e63-ba5f-64909819b67d",
        "device_id": "ca614e6e-bd7c-4f81-b95b-9527a84b9aeb"
    },
    {
        "user_name": "Marie",
        "email": "marie@example.com",
        "device_name": "Marie Laptop",
        "device_created": "2024-08-30 20:44:54",
        "user_id": "4b7706f1-d702-4e63-ba5f-64909819b67d",
        "device_id": "f53e7bc8-4225-4a86-a937-80e66779a1aa"
    }
]


In [52]:
# List Everything
list_everything_joined()

,id,name,email,password,user_id,device_id,id,name,created_at
0,f84c183c-a3f0-4f02-bc06-acd91ca1d048,Bob,bob@example.com,1234,f84c183c-a3f0-4f02-bc06-acd91ca1d048,ec1846b5-6b82-4e87-b553-cf0c8e36fe61,ec1846b5-6b82-4e87-b553-cf0c8e36fe61,Bob Laptop,2024-08-30 20:44:54
1,f84c183c-a3f0-4f02-bc06-acd91ca1d048,Bob,bob@example.com,1234,f84c183c-a3f0-4f02-bc06-acd91ca1d048,b373cf29-54d1-497a-9843-e81fce68a2b0,b373cf29-54d1-497a-9843-e81fce68a2b0,Bob Phone,2024-08-30 20:44:54
2,c758ce01-fa8b-4f4f-8856-a2c79d1a636c,James,james@example.com,7654,c758ce01-fa8b-4f4f-8856-a2c79d1a636c,7ff7cb19-5356-4d73-8155-bea6484e7562,7ff7cb19-5356-4d73-8155-bea6484e7562,James Laptop,2024-08-30 20:44:54
3,c758ce01-fa8b-4f4f-8856-a2c79d1a636c,James,james@example.com,7654,c758ce01-fa8b-4f4f-8856-a2c79d1a636c,3e139f88-e54a-419d-ba9c-84071115da19,3e139f88-e54a-419d-ba9c-84071115da19,James Phone,2024-08-30 20:44:54
4,4b7706f1-d702-4e63-ba5f-64909819b67d,Marie,marie@example.com,0000,4b7706f1-d702-4e63-ba5f-64909819b67d,f53e7bc8-4225-4a86-a937-80e66779a1aa,f53e7bc8-4225-4a86-a937-80e66779a1aa,Marie Laptop,2024-08-30 20:44:54
5,4b7706f1-d702-4e63-ba5f-64909819b67d,Marie,marie@example.com,0000,4b7706f1-d702-4e63-ba5f-64909819b67d,ca614e6e-bd7c-4f81-b95b-9527a84b9aeb,ca614e6e-bd7c-4f81-b95b-9527a84b9aeb,Marie Phone,2024-08-30 20:44:54


Notice how there are multiple "id" and "name" columns since they were not given different names in the SELECT statement. Lets see if that will be an issue.

In [53]:
list_everything_joined()['id']

,id,id
0,f84c183c-a3f0-4f02-bc06-acd91ca1d048,ec1846b5-6b82-4e87-b553-cf0c8e36fe61
1,f84c183c-a3f0-4f02-bc06-acd91ca1d048,b373cf29-54d1-497a-9843-e81fce68a2b0
2,c758ce01-fa8b-4f4f-8856-a2c79d1a636c,7ff7cb19-5356-4d73-8155-bea6484e7562
3,c758ce01-fa8b-4f4f-8856-a2c79d1a636c,3e139f88-e54a-419d-ba9c-84071115da19
4,4b7706f1-d702-4e63-ba5f-64909819b67d,f53e7bc8-4225-4a86-a937-80e66779a1aa
5,4b7706f1-d702-4e63-ba5f-64909819b67d,ca614e6e-bd7c-4f81-b95b-9527a84b9aeb


Yes it absolutely will be. Which one belongs to which table? The solution is to give the columns different names in the SELECT statement like in ```list_devices_for_user()```.

## Bonus: Export to Excel

Usually this would be a very bad idea for a production database, but in this case it might be helpful to export the data to an excel file to further understand how it works. Run the code below, install the [Office Viewer VSCode Extension](https://marketplace.visualstudio.com/items?itemName=cweijan.vscode-office) and open the [sqlite_database.xlsx](sqlite_database.xlsx) file. There will be one sheet per table at the bottom of the document.

In [54]:
# Export everything to excel
users_df = list_users()
devices_df = list_devices()
user_devices_df = list_user_devices_table()

with pd.ExcelWriter('database_dump.xlsx') as writer:
    users_df.to_excel(writer, sheet_name='users', index=False)
    devices_df.to_excel(writer, sheet_name='devices', index=False)
    user_devices_df.to_excel(writer, sheet_name='user_devices', index=False)

## IMPORTANT: Closing the connection/Releasing resources

At the start of this notebook, you created a connection to the database. An SQL connection should always be closed when it is no longer needed. To see why, open the [folder](.) and try to delete the [sqlite_database.db](sqlite_database.db) file. You will not be able to because the connection is still open, meaning the resource is still "locked". Run the code below to close the connection, also known as releasing/freeing the resource, and then try to delete the file again.

In [55]:
connection.close()